# core

> Selection step state management helpers

In [ ]:
#| default_exp routes.core

In [ ]:
#| export
from typing import List, Dict, Any, Optional, Tuple, Union

from fasthtml.common import Div, Script

from cjm_workflow_state.state_store import SQLiteWorkflowStateStore

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.html_ids import SelectionHtmlIds
from cjm_transcript_source_select.components.source_browser import (
    _render_source_list
)
from cjm_transcript_source_select.components.selection_queue import (
    _render_selection_queue
)
from cjm_transcript_source_select.components.step_renderer import (
    _render_selection_stats
)
from cjm_transcript_source_select.services.source import SourceService

# Debug flag for selection state tracing (set False in production)
DEBUG_SELECTION_STATE = False

# Type alias for state store (duck-typed, accepts any implementation with get_state/update_state)
WorkflowStateStore = SQLiteWorkflowStateStore

## State Management Helpers

These helpers provide typed access to the selection step state in route handlers and duplicate audio source prevention.

In [ ]:
#| export
def _get_step_state(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str  # Session identifier string
) -> Dict[str, Any]:  # Step state dictionary
    """Get the selection step state from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("selection", {})

def _find_duplicate_media_source(
    source_service: SourceService,  # Source service for lookups
    record_id: str,  # Candidate record ID
    provider_id: str,  # Candidate provider ID
    selected_sources: List[Dict[str, str]],  # Current selections
) -> Optional[Dict[str, str]]:  # Conflicting source dict or None
    """Find an already-selected source that shares the same audio file."""
    candidate = source_service.get_transcription_by_id(record_id, provider_id)
    if not candidate or not candidate.media_path:
        return None
    for s in selected_sources:
        existing = source_service.get_transcription_by_id(s["record_id"], s["provider_id"])
        if existing and existing.media_path == candidate.media_path:
            return s
    return None

def _render_duplicate_flash(
    candidate_record_id: str,  # Record ID of the row the user clicked
    candidate_provider_id: str,  # Provider ID of the row the user clicked
    existing_record_id: str,  # Record ID of the conflicting selected row
    existing_provider_id: str,  # Provider ID of the conflicting selected row
) -> Div:  # OOB Div with flash script (replaces previous via innerHTML swap)
    """Render a fixed-ID container with flash script for two source rows."""
    row1 = SelectionHtmlIds.source_row(candidate_record_id, candidate_provider_id)
    row2 = SelectionHtmlIds.source_row(existing_record_id, existing_provider_id)
    return Div(
        Script(f"""
(function() {{
    function flash() {{
        var r1 = document.getElementById('{row1}');
        var r2 = document.getElementById('{row2}');
        [r1, r2].forEach(function(r) {{
            if (r) {{
                r.style.transition = 'background-color 0.05s ease-in';
                r.classList.add('bg-error');
            }}
        }});
        setTimeout(function() {{
            [r1, r2].forEach(function(r) {{
                if (r) {{
                    r.style.transition = 'background-color 0.3s ease-out';
                    r.classList.remove('bg-error');
                }}
            }});
            setTimeout(function() {{
                [r1, r2].forEach(function(r) {{
                    if (r) {{ r.style.transition = ''; }}
                }});
            }}, 300);
        }}, 600);
    }}
    // Defer until after HTMX finishes all OOB swaps
    document.body.addEventListener('htmx:afterSettle', function handler() {{
        document.body.removeEventListener('htmx:afterSettle', handler);
        flash();
    }});
}})();
"""),
        id=SelectionHtmlIds.SCRIPT_RUNNER,
        hx_swap_oob="innerHTML",
    )

In [ ]:
#| export
def _get_active_source_tab(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str  # Session identifier string
) -> str:  # Active tab: "db" or "files"
    """Get the currently active source tab from workflow state."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    return workflow_state.get("source_tab", "db")

## Queue Response Builder

Shared response builder for handlers that mutate the selection queue. Covers three patterns:

- **Pattern A** (default): queue + OOB stats + conditional OOB source list
- **Pattern B** (`include_stats=False, include_source_list=False`): queue only
- **Pattern C** (`include_stats=False`): queue + conditional OOB source list

In [ ]:
#| export
def _build_queue_response(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for querying transcriptions
    session_id: str,  # Session identifier string
    selected_sources: List[Dict[str, str]],  # Current selected sources after mutation
    urls: SelectionUrls,  # URL bundle for rendering
    include_stats: bool = True,  # Include OOB stats swap
    include_source_list: bool = True,  # Include conditional OOB source list swap
    grouping_mode: str = None,  # Override grouping mode for source list rendering
) -> Union[Any, Tuple]:  # Single component or tuple of components with OOB swaps
    """Build the standard response for queue-mutating handlers."""
    queue = _render_selection_queue(
        selected_sources=selected_sources,
        remove_url=urls.remove,
        reorder_url=urls.reorder,
        clear_url=urls.clear,
    )

    parts = [queue]

    # Lazy-load transcriptions only when needed for stats or source list
    all_transcriptions = None
    if include_stats or include_source_list:
        all_transcriptions = source_service.query_transcriptions(limit=500)

    if include_stats:
        parts.append(_render_selection_stats(
            selected_sources=selected_sources,
            transcriptions=all_transcriptions,
            oob=True,
        ))

    if include_source_list and _get_active_source_tab(state_store, workflow_id, session_id) == "db":
        source_list_kwargs = {}
        if grouping_mode is not None:
            source_list_kwargs["grouping_mode"] = grouping_mode
        parts.append(_render_source_list(
            transcriptions=all_transcriptions,
            selected_sources=selected_sources,
            add_url=urls.add,
            remove_url=urls.remove,
            preview_url=urls.preview,
            select_all_url=urls.select_all,
            oob=True,
            **source_list_kwargs,
        ))

    return tuple(parts) if len(parts) > 1 else parts[0]

In [ ]:
#| export
def _update_step_state(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str,  # Session identifier string
    selected_sources: List[Dict[str, str]] = None,  # Updated selected sources list (None = don't change)
    grouping_mode: str = None,  # Updated grouping mode (None = don't change)
    external_db_paths: List[str] = None,  # Updated external db paths (None = don't change)
    current_browse_path: str = None,  # Updated current browse path (None = don't change)
    file_browser_state: Dict[str, Any] = None,  # Updated file browser state (None = don't change)
) -> None:
    """Update the selection step state in the workflow state store."""
    if DEBUG_SELECTION_STATE:
        print(f"[SELECTION_STATE] _update_step_state called")
        if selected_sources is not None:
            print(f"[SELECTION_STATE] Setting selected_sources count = {len(selected_sources)}")

    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    selection_state = step_states.get("selection", {})
    
    # Update only the fields that are provided
    if selected_sources is not None:
        selection_state["selected_sources"] = selected_sources
    if grouping_mode is not None:
        selection_state["grouping_mode"] = grouping_mode
    if external_db_paths is not None:
        selection_state["external_db_paths"] = external_db_paths
    if current_browse_path is not None:
        selection_state["current_browse_path"] = current_browse_path
    if file_browser_state is not None:
        selection_state["file_browser_state"] = file_browser_state
    
    step_states["selection"] = selection_state
    workflow_state["step_states"] = step_states

    if DEBUG_SELECTION_STATE:
        print(f"[SELECTION_STATE] Calling update_state with step_states keys = {list(step_states.keys())}")

    state_store.update_state(workflow_id, session_id, workflow_state)

    if DEBUG_SELECTION_STATE:
        # Verify what was stored
        verify_state = state_store.get_state(workflow_id, session_id)
        verify_selection = verify_state.get('step_states', {}).get('selection', {})
        print(f"[SELECTION_STATE] VERIFY: selection.selected_sources count = {len(verify_selection.get('selected_sources', []))}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()